In [1]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split
import torchvision
from torchvision import transforms
from torchvision.models import efficientnet_v2_s, EfficientNet_V2_S_Weights
import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
import wandb

from utils import *

In [2]:
NUM_CLASSES = 10
BATCH_SIZE = 32
NUM_WORKERS = 4
MAX_EPOCHS = 10
LEARNING_RATE = 3e-4
FEATURE_LEARNING_RATE = 1e-5

In [3]:
class FineTunedEfficientNet(pl.LightningModule):
    def __init__(self, num_classes=10, learning_rate=3e-4, feature_learning_rate=1e-5):
        super().__init__()
        self.learning_rate = learning_rate
        self.feature_learning_rate = feature_learning_rate
        
        # Load pre-trained model
        self.model = efficientnet_v2_s(weights=EfficientNet_V2_S_Weights.IMAGENET1K_V1)
        
        # Freeze early layers (first 4 blocks)
        for i, block in enumerate(self.model.features):
            if i < 4:  # Freeze first 4 blocks
                for param in block.parameters():
                    param.requires_grad = False
        
        # Replace classifier with new one for our classes
        num_features = self.model.classifier[1].in_features
        self.model.classifier[1] = nn.Linear(num_features, num_classes)
        
        # Save hyperparameters for checkpointing
        self.save_hyperparameters()
    
    def forward(self, x):
        return self.model(x)
    
    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)
        
        # Log metrics
        preds = torch.argmax(logits, dim=1)
        acc = (preds == y).float().mean()
        self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True)
        self.log('train_acc', acc, on_step=True, on_epoch=True, prog_bar=True)
        
        return loss
    
    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)
        
        # Log metrics
        preds = torch.argmax(logits, dim=1)
        acc = (preds == y).float().mean()
        self.log('val_loss', loss, prog_bar=True)
        self.log('val_acc', acc, prog_bar=True)
        
        return loss
    
    def test_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y)
        
        # Log metrics
        preds = torch.argmax(logits, dim=1)
        acc = (preds == y).float().mean()
        self.log('test_loss', loss)
        self.log('test_acc', acc)
        
        return loss
    
    def configure_optimizers(self):
        # Use different learning rates for pre-trained feature layers and new classifier
        classifier_params = self.model.classifier.parameters()
        feature_params = self.model.features.parameters()
        
        optimizer = torch.optim.AdamW([
            {'params': classifier_params, 'lr': self.learning_rate},
            {'params': feature_params, 'lr': self.feature_learning_rate}
        ])
        
        # Learning rate scheduler
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
            optimizer, mode='min', factor=0.5, patience=3, verbose=True
        )
        
        return {
            'optimizer': optimizer,
            'lr_scheduler': {
                'scheduler': scheduler,
                'monitor': 'val_loss',
                'interval': 'epoch'
            }
        }

In [4]:
# Initialize WandB
wandb.init(project="inaturalist-comparison")

data_directory = r'C:\Users\DELL\Desktop\Coding\Python\DL\Assignment 2\da6401_assignment2\data\inaturalist_12K'
data_module = iNaturalistDataModule(
        data_dir=data_directory,
        batch_size=BATCH_SIZE,
        use_data_augmentation=True
    )
data_module.setup()

# Setup callbacks
checkpoint_callback = ModelCheckpoint(
    monitor='val_acc',
    dirpath='checkpoints/',
    filename='inaturalist-{epoch:02d}-{val_acc:.2f}',
    save_top_k=3,
    mode='max',
)

early_stop_callback = EarlyStopping(
    monitor='val_loss',
    patience=5,
    mode='min'
)

wandb_logger = WandbLogger(project="inaturalist-cnn")

# Train fine-tuned model
print("Training fine-tuned EfficientNetV2...")
fine_tuned_model = FineTunedEfficientNet(
    num_classes=NUM_CLASSES,
    learning_rate=LEARNING_RATE,
    feature_learning_rate=FEATURE_LEARNING_RATE
)

trainer_fine_tuned = pl.Trainer(
    max_epochs=MAX_EPOCHS,
    callbacks=[checkpoint_callback, early_stop_callback],
    logger=wandb_logger,
    log_every_n_steps=10,
    deterministic=True
)


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: bullseye2608 (bullseye2608-indian-institute-of-technology-madras) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Training fine-tuned EfficientNetV2...


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [5]:

trainer_fine_tuned.fit(fine_tuned_model, data_module)


You are using a CUDA device ('NVIDIA GeForce RTX 3060 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
c:\Users\DELL\.conda\envs\DL\lib\site-packages\pytorch_lightning\loggers\wandb.py:397: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
c:\Users\DELL\.conda\envs\DL\lib\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:654: Checkpoint directory C:\Users\DELL\Desktop\Coding\Python\DL\Assignment 2\da6401_assignment2\partB\checkpoints exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
c:\Users\DELL\.conda\envs\DL\lib\site-packages\torch\optim\lr_scheduler.

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.


In [11]:

# Test fine-tuned model
fine_tuned_results = trainer_fine_tuned.test(fine_tuned_model, data_module)

# Now train the model from scratch
wandb.finish()  # End the previous run

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │    0.8669999837875366     │
│         test_loss         │    0.49156495928764343    │
└───────────────────────────┴───────────────────────────┘

epoch,▁▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇█████
test_acc,▁
test_loss,▁
train_acc_epoch,▁▅▆▆▆▇▇███
train_acc_step,▁▄▅▄▆▇▆▆▆▅▅▆▇▇▆▆▇▆▇▆█▆▇▇▆█▆▇▇▇▆▇▇▇█▇▇▇▇█
train_loss_epoch,█▄▃▃▂▂▂▁▁▁
train_loss_step,█▆▅▄▃▄▂▃▂▂▂▂▂▄▂▂▁▂▂▂▃▂▂▂▁▂▂▂▂▁▂▂▂▂▁▂▂▁▂▁
trainer/global_step,▁▁▁▁▂▂▂▃▃▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇▇██
val_acc,▁▄▅▆▆▇▇▇██
val_loss,▂▂▁▁▁▁▁▁▁█
epoch,10


In [12]:

import numpy as np
import matplotlib.pyplot as plt
import wandb

def create_prediction_grid(model, test_dataset, class_names):
    # Select 30 random images (for a 10×3 grid)
    num_samples = min(30, len(test_dataset))
    indices = np.random.choice(len(test_dataset), num_samples, replace=False)
    
    # Create a figure with 10 rows and 3 columns
    fig, axes = plt.subplots(10, 3, figsize=(12, 30))
    
    # Define denormalization transform
    denorm = transforms.Compose([
        transforms.Normalize(mean=[0, 0, 0], std=[1/0.229, 1/0.224, 1/0.225]),
        transforms.Normalize(mean=[-0.485, -0.456, -0.406], std=[1, 1, 1]),
    ])
    
    # Plot each image with prediction
    model.to('cuda' if torch.cuda.is_available() else 'cpu')
    model.eval()
    
    with torch.no_grad():
        for i, idx in enumerate(indices):
            row = i // 3
            col = i % 3
            
            # Get the image and label
            img, true_label = test_dataset[idx]
            img_tensor = img.unsqueeze(0).to(model.device)
            
            # Make prediction
            output = model(img_tensor)
            _, pred_label = torch.max(output, 1)
            
            # Get probabilities
            probs = torch.nn.functional.softmax(output, dim=1)[0]
            
            # Convert image for display
            img_display = denorm(img).permute(1, 2, 0).cpu().numpy()
            img_display = np.clip(img_display, 0, 1)
            
            # Get prediction info
            true_class = class_names[true_label]
            pred_class = class_names[pred_label.item()]
            is_correct = true_label == pred_label.item()
            
            # Plot image
            ax = axes[row, col]
            ax.imshow(img_display)
            
            # Set the title color based on correct/wrong prediction
            title_color = 'green' if is_correct else 'red'
            
            # Add title with prediction and confidence
            conf = probs[pred_label.item()].item() * 100
            ax.set_title(f"True: {true_class}\nPred: {pred_class}\nConf: {conf:.1f}%", 
                        color=title_color, fontsize=10)
            
            # Remove ticks
            ax.set_xticks([])
            ax.set_yticks([])
    
    # Add overall title
    # plt.suptitle("Model Predictions on Test Data", fontsize=16)
    plt.tight_layout()
    
    # Save the figure
    plt.savefig('prediction_grid.png', dpi=300, bbox_inches='tight')
    
    # Log the figure to wandb
    wandb.log({"Model Predictions on Test Data": wandb.Image(fig)})
    
    # Close the figure
    plt.close(fig)

In [ ]:
wandb.init(project="inaturalist-cnn")
create_prediction_grid(fine_tuned_model, data_module.test_dataset, data_module.test_dataloader().dataset.classes)
wandb.finish()

In [1]:
import wandb
from datetime import datetime

timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
run_name = f"ImageLogger_{timestamp}"
# Initialize W&B run
run = wandb.init(project="inaturalist-cnn", name=run_name)

# Log the PNG image
image = wandb.Image("./prediction_grid.png", caption="Model Predictions on Test Data")
run.log({"Model Predictions on Test Data": image})

# Finish the run
run.finish()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: bullseye2608 (bullseye2608-indian-institute-of-technology-madras) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
